In [1]:
#### 利益改善され、割安な銘柄が動意づいたときに検知することを目的とする

In [2]:
# 読み込みファイルパスの設定とimportしたいmoduleパス(pythonパス)の設定
from pathlib import Path
import os, sys

CURRENT_DIR = Path(os.getcwd())
PJ_DIR = CURRENT_DIR.parent.parent
LIB_DIR = PJ_DIR / "lib"
DATA_DIR = PJ_DIR / "data" 

sys.path.append(str(LIB_DIR))

# notebook内で利用するmoduleのimport
from lib_dataprocess import KessanPl, PricelistFig, PricelistPl, MeigaralistPl, read_data
from lib_dataprocess import ShikihoPl, FinancequotePl, print_finance_quote, PricelistFig, KessanFig
import polars as pl
from datetime import date
from dateutil.relativedelta import relativedelta

fp = DATA_DIR/"reviced_pricelist.parquet"
df = read_data(fp)
RevPl = PricelistPl(df)

# dataの読み込み
fp = DATA_DIR/"raw_pricelist.parquet"
df = read_data(fp)
PPL = PricelistPl(df)

fp = DATA_DIR/"finance_quote.parquet"
df = read_data(fp)
FPL = FinancequotePl(df)
FPL.with_columns_market_cap()

fp = DATA_DIR/"meigaralist.parquet"
df = read_data(fp)
MPL = MeigaralistPl(df)

fp = DATA_DIR/"shikiho.parquet"
df = read_data(fp)
SPL = ShikihoPl(df)


In [3]:
#####
##### KessanPl.dfの加工
#####

In [4]:
#パラメータのセット
# 決算日の指定 -> 指定した日の四半期決算でランキングを作成する
valuation_date = date.today()
valuation_date = date(2024, 9, 26)
# 売上高伸び率を何%以上にするか
min_sales_growthrate = 10

In [5]:
# 読み込みと加工
# 読み込み
fp = DATA_DIR/"kessan.parquet"
df = read_data(fp)
KPL1 = KessanPl(df)
KPL2 = KessanPl(df)

# データ加工
KPL1.with_columns_growthrate_lastyear()
KPL2.with_columns_diff_growthrate()
KPL1.df = KPL1.df.filter(pl.col("settlement_type")=="四")
KPL2.df = KPL2.df.filter(pl.col("settlement_type")=="四")
df = KPL1.df.join(KPL2.df, on=["code", "settlement_date"], how="left")
df = df.select([
    "code", "settlement_date", "announcement_date", "sales", "operating_income", "sales_growthrate", "diff_operating_income_growthrate"
])
# valuation_date日時点の最新データに絞る
df = df.filter(pl.col("announcement_date")<=valuation_date)
df = df.group_by("code").agg([
    pl.col("settlement_date").last().alias("settlement_date"),
    pl.col("announcement_date").last().alias("announcement_date"),
    pl.col("sales").last().alias("sales"),
    pl.col("operating_income").last().alias("operating_income"),
    pl.col("sales_growthrate").last().alias("sales_growthrate"),
    pl.col("diff_operating_income_growthrate").last().alias("diff_operating_income_growthrate")
])

target_date = valuation_date - relativedelta(days=90)
df = df.filter(pl.col("announcement_date")>target_date)

cpl_df = df
cpl_df

code,settlement_date,announcement_date,sales,operating_income,sales_growthrate,diff_operating_income_growthrate
i64,date,date,i64,i64,f64,f64
1301,2024-06-30,2024-08-05,68204,3102,9.4,11.7
1332,2024-06-30,2024-08-06,220644,9724,10.4,0.0
1333,2024-06-30,2024-08-05,256928,7657,2.4,-0.5
1375,2024-06-30,2024-08-08,10172,-77,22.9,12.5
1376,2024-05-31,2024-07-10,23224,1150,1.9,-16.0
…,…,…,…,…,…,…
9991,2024-06-30,2024-07-25,26954,1366,-14.5,-2.5
9993,2024-05-31,2024-07-11,24969,-408,1.8,-108.5
9994,2024-06-30,2024-08-09,39151,1286,0.6,-14.9


In [6]:
##### 
##### reviced_pricelistの加工
#####

In [7]:
#パラメータのセット
min_zs25 = 2
min_daily_updown_rate = 2
min_overnight_updown_rate = 0

In [8]:
# 読み込みと加工
# 読み込み
fp = DATA_DIR/"reviced_pricelist.parquet"
df = read_data(fp)
RPL = PricelistPl(df)

# データ加工
# rpl_dfに評価列を追加
RPL.with_columns_moving_zscore()
RPL.with_columns_daily_updown_rate()
RPL.with_columns_overnight_updown("yesterday")
rpl_df = RPL.df
# rpl_dfの評価対象行を抽出
ori_cols = rpl_df.columns
df = cpl_df
df = df.select(["code", "announcement_date"])
df = df.with_columns([
    pl.col("announcement_date").alias("start_date"),
    (pl.col("announcement_date") + pl.duration(days=93)).alias("end_date")
])
df = df.select(["code", "start_date", "end_date"])
df = rpl_df.join(df, on="code", how="left")
df = df.filter(pl.col("date")>=pl.col("start_date"))\
    .filter(pl.col("date")<=pl.col("end_date"))
df = df.select(ori_cols)
rpl_df = df
rpl_df

code,date,open,high,low,close,volume,accumulated_split_rate,zs25,daily_updown_rate,overnight_updown_rate
i64,date,f64,f64,f64,f64,f64,f64,f64,f64,f64
1301,2024-08-05,3610.0,3780.0,3400.0,3450.0,105000.0,1.0,4.2,-4.43,-3.99
1301,2024-08-06,3855.0,3855.0,3605.0,3665.0,50100.0,1.0,1.2,-4.93,11.74
1301,2024-08-07,3630.0,3755.0,3600.0,3650.0,38600.0,1.0,0.58,0.55,-0.95
1301,2024-08-08,3635.0,3690.0,3630.0,3645.0,23400.0,1.0,-0.23,0.28,-0.41
1301,2024-08-09,3690.0,3720.0,3630.0,3675.0,31700.0,1.0,0.17,-0.41,1.23
…,…,…,…,…,…,…,…,…,…,…
9997,2024-10-28,702.0,711.0,699.0,708.0,154300.0,1.0,-0.59,0.85,0.86
9997,2024-10-29,711.0,722.0,710.0,716.0,205600.0,1.0,-0.39,0.7,0.42
9997,2024-10-30,715.0,718.0,711.0,712.0,208800.0,1.0,-0.33,-0.42,-0.14


In [9]:
##### 
##### cpl_dfとrpl_dfを連結して、screening用のdfを作成
#####

In [10]:
df = rpl_df.join(cpl_df, on="code", how="left")
screened_df = df
screened_df

code,date,open,high,low,close,volume,accumulated_split_rate,zs25,daily_updown_rate,overnight_updown_rate,settlement_date,announcement_date,sales,operating_income,sales_growthrate,diff_operating_income_growthrate
i64,date,f64,f64,f64,f64,f64,f64,f64,f64,f64,date,date,i64,i64,f64,f64
1301,2024-08-05,3610.0,3780.0,3400.0,3450.0,105000.0,1.0,4.2,-4.43,-3.99,2024-06-30,2024-08-05,68204,3102,9.4,11.7
1301,2024-08-06,3855.0,3855.0,3605.0,3665.0,50100.0,1.0,1.2,-4.93,11.74,2024-06-30,2024-08-05,68204,3102,9.4,11.7
1301,2024-08-07,3630.0,3755.0,3600.0,3650.0,38600.0,1.0,0.58,0.55,-0.95,2024-06-30,2024-08-05,68204,3102,9.4,11.7
1301,2024-08-08,3635.0,3690.0,3630.0,3645.0,23400.0,1.0,-0.23,0.28,-0.41,2024-06-30,2024-08-05,68204,3102,9.4,11.7
1301,2024-08-09,3690.0,3720.0,3630.0,3675.0,31700.0,1.0,0.17,-0.41,1.23,2024-06-30,2024-08-05,68204,3102,9.4,11.7
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
9997,2024-10-28,702.0,711.0,699.0,708.0,154300.0,1.0,-0.59,0.85,0.86,2024-06-30,2024-07-31,51368,982,-0.9,36.7
9997,2024-10-29,711.0,722.0,710.0,716.0,205600.0,1.0,-0.39,0.7,0.42,2024-06-30,2024-07-31,51368,982,-0.9,36.7
9997,2024-10-30,715.0,718.0,711.0,712.0,208800.0,1.0,-0.33,-0.42,-0.14,2024-06-30,2024-07-31,51368,982,-0.9,36.7


In [11]:
##### 
##### screening
#####

In [12]:
# para
min_zs25 = 3
min_sales = 1000
min_daily_updown_rate = 2
min_overnight_updown_rate = 0
min_sales_growthrate = 10
min_sgr_dogr_ratio = 2

In [64]:
df = screened_df

df = df.filter([
    pl.col("zs25")>=min_zs25,
    pl.col("sales")>=min_sales,
    pl.col("daily_updown_rate")>=min_daily_updown_rate,
    pl.col("overnight_updown_rate")>=min_overnight_updown_rate,
    pl.col("sales_growthrate")>=min_sales_growthrate,
    pl.lit(min_sgr_dogr_ratio) * pl.col("sales_growthrate")<pl.col("diff_operating_income_growthrate")
])
df[30:40]

code,date,open,high,low,close,volume,accumulated_split_rate,zs25,daily_updown_rate,overnight_updown_rate,settlement_date,announcement_date,sales,operating_income,sales_growthrate,diff_operating_income_growthrate
i64,date,f64,f64,f64,f64,f64,f64,f64,f64,f64,date,date,i64,i64,f64,f64
4478,2024-10-03,2504.0,2755.0,2460.0,2744.0,1.4868e6,1.0,3.88,9.58,3.05,2024-06-30,2024-08-14,7007,-1426,29.6,111.8
4490,2024-07-17,1332.0,1429.0,1251.0,1392.0,619900.0,1.0,4.73,4.5,2.3,2024-05-31,2024-07-12,2520,385,18.1,121.0
4490,2024-07-22,1270.0,1454.0,1250.0,1410.0,680200.0,1.0,3.37,11.02,2.42,2024-05-31,2024-07-12,2520,385,18.1,121.0
4506,2024-08-19,564.0,612.0,551.0,593.0,1.34777e7,1.0,3.09,5.14,1.81,2024-06-30,2024-07-31,90675,-3105,19.8,323.3
4540,2024-09-24,4125.0,4254.0,4105.0,4241.0,759400.0,1.0,4.06,2.81,3.88,2024-06-30,2024-08-05,43690,10575,18.0,88.5
4540,2024-09-26,4400.0,4530.0,4381.0,4521.0,869800.0,1.0,3.13,2.75,1.36,2024-06-30,2024-08-05,43690,10575,18.0,88.5
4704,2024-08-13,8000.0,8451.0,7973.0,8235.0,4.1664e6,1.0,4.37,2.94,3.33,2024-06-30,2024-08-08,68603,12309,12.8,47.2
4716,2024-09-26,14275.0,15190.0,14240.0,15190.0,504400.0,1.0,4.19,6.41,4.24,2024-08-31,2024-09-25,63915,22194,11.4,56.9
4792,2024-08-07,2033.0,2145.0,1986.0,2120.0,123000.0,1.0,4.22,4.28,14.66,2024-06-30,2024-08-06,7269,1978,34.0,74.5


In [65]:
##### 
##### individual stock info
#####

In [69]:
# set code
code = 4478
marked_day = date(2024, 10, 3)

In [70]:
### 最新情報
# 指定したcodeの指定した日における株価と各種ファンダメンタルズデータをまとめて標準出力する
# pricelist_dfは、raw_pricelistかreviced_pricelistかケースに応じて使い分ける。
# def print_finance_quote(
#        pricelist_df: pl.DataFrame,
#        finance_quote_df: pl.DataFrame,
#        code: int, 
#        valuation_date: date=date.today()
#    ) -> None:
print_finance_quote(PPL.df, FPL.df, code, valuation_date=marked_day)
print()
SPL.print_stock_df(code, num=4, valuation_date=valuation_date)

フリー(4478)の銘柄情報

終値: 2744.0円(2024年10月03日)
予想配当利回り: 0.0%(2024年10月03日)
予想PER: None倍(2024年10月03日)
実績PBR: 9.58倍(2024年10月03日)
自己資本比率: 42.0%(2024年10月03日)
予想ROE: 決算予想がないため、表示不可
予想ROA: 決算予想がないため、表示不可
時価総額: 1608.3億円(2024年10月03日)

4478(フリー)の四季報データ履歴

発行日: 2024年09月13日
【黒字近づく】
  柱のERPは中堅企業の追加契約が順調に推移、一部製品の値上げも寄与。個人向け堅調増。増収効果と広告宣伝費圧縮、開発費の資産計上などで営業赤字幅が急減。買収時の一括減損特損消える。
【効率化】
  請求書や法人カードの従量課金型、顧客自身による追加購入など営業の効率化を模索。制度対応が一服、蓄積した見込み客への再提案に重点置いて生産性向上。

発行日: 2024年06月14日
【赤字激減】
  高単価な中堅企業向けが順調増。人員増や広告費増などの先行費用重い。25年6月期は中堅企業の追加契約が大幅増。費用増も徹底的に抑制し、営業赤字幅は急縮小。買収関連の減損特損消える。
【転換点】
  25年6月期に調整後利益で上場後初の黒字化が視野。開発費の資産計上開始、固定資産や買収時のれんの毎四半期減損処理も終了。7月に会計製品を価格改定。

発行日: 2024年03月16日
【赤字続く】
  制度改正を背景に高単価の中堅企業向けが順調増。ただ人員増や広告費増などの先行費用重い。なお営業赤字。買収関連特損。25年6月期は追加契約が大幅増。費用増負担も一服。赤字幅急縮小。
【借り入れ】
  25年6月期に調整後利益で黒字化視野。32億円を借り入れ、カード事業強化。決済から証憑回収、仕訳まで一貫提供。山梨中央銀行と提携、域内DX推進。

発行日: 2023年12月15日
【費用先行】
  中堅企業向けの開拓進み、クラウド型ERPの新規利用者が順調増。ただ、人員や広告費などの費用増重い。大幅増収ながら営業赤字やや拡大。買収関連特損、案件増えれば最終赤字幅は一段増す。
【取得】
  エン・ジャパンからフリーランス管理ツール事業を取得。同分野向けの受発注管理を強化

In [71]:
KFIG = KessanFig(code, "四半期", end_settlement_date=valuation_date)
KFIG.add_trace_profits()
KFIG.fig.show()

In [72]:
print(valuation_date)
delta = relativedelta(days=90)
start_date = marked_day - delta
end_date = marked_day + delta
PFIG = PricelistFig(code, RevPl.df, MPL.df, start_date, end_date)
df = RevPl.df.filter(pl.col("code")==code).filter(pl.col("date")<=marked_day)
vline_date = df["date"].to_list()[-1]
# screeningで抽出した日にvlineを引く 
PFIG.add_vline(marked_day)
# 決算発表日にvlineを引く
PFIG.add_vline_announcement_date()
PFIG.fig.show()

2024-09-26


In [63]:
fp = DATA_DIR/"kessan.parquet"
df = read_data(fp)
KPL = KessanPl(df)

KPL.df.filter(pl.col("code")==code).filter(pl.col("announcement_date")>=start_date).filter(pl.col("announcement_date")<=end_date)

code,settlement_date,settlement_type,announcement_date,sales,operating_income,ordinary_profit,final_profit,reviced_eps,dividend,quater
i64,date,str,date,i64,i64,i64,i64,f64,f64,i64
4403,2024-03-31,"""本""",2024-05-09,222252,42142,45577,33990,141.2,38.0,4
4403,2024-03-31,"""四""",2024-05-09,62153,8880,9844,7979,33.1,14.3,4
4403,2025-03-31,"""予""",2024-05-09,232000,40500,41600,33000,138.4,40.0,4
4403,2024-06-30,"""四""",2024-08-07,55110,13533,15160,11047,46.3,24.6,1
